### usage example - PuO2

In [ ]:
!pip install mlipts

### Part I

First we set up a number of LAMMPS calculations for defining a configuration space.

For this to work you must have the directory ```lammps_base_*``` in your working directory. These settings are copied to all lammps caculations. An example is given for PuO2

Notice in in.PuO2 there is a variable lammps will not regonise: ``` £T ```. The code regonises ```£``` as a variable and will generate a directory for each value set in a corresponding variable dict.


In [1]:
from mlipts.data_collection import DataCollection

# initialize with atom types in desired order
workflow = DataCollection(['Pu','O'])

# define £T values

variables = { '£T' : [100,200] }

workflow.build_MD_calculations('./lammps_base_PuO2',variables,outdir='./MD_calculations')

Caculation directory generated: ./MD_calculations/./MD_calculations/lammps_T_100
Caculation directory generated: ./MD_calculations/./MD_calculations/lammps_T_200


now create a script to run these calculations sequentially. For this you need to specify a lammps command line.

In [ ]:

lammps_exec = "path/to/lammps/build/lmp"  # replace with actual path to LAMMPS executable

lammps_cmd_line = "srun --distribution=block:block --hint=nomultithread " + lammps_exec + " -i in.PuO2" + " -l out.PuO2"

workflow.write_MD_submission_scripts(MD_cmd_line=lammps_cmd_line,
                                     nodes=1,ranks=1,time='00:20:00',
                                     hpc='archer2',
                                     hpc_account='my_account',
                                     submit=True,
                                     npartitions=1)

MD submission script saved to: ./MD_scripts/MD_submission_script_#0
Submitted batch job 11867600


### Part II

Often you may return to the workflow after some time, while your calculations were running and the kernel has restarted. If this is the case you will need to create a new DataCollection object and use the ```set_active_MD_dirs()``` method.

In [1]:
# only if kernel restarted

from mlipts.data_collection import DataCollection
workflow = DataCollection(['Pu','O'])
workflow.set_active_MD_dirs('MD_calculations')
# check if correct.
workflow.check_active_MD()

Number of active MD directories updated from 0 to 2
MD directories currently active: 
['MD_calculations/lammps_T_100', 'MD_calculations/lammps_T_200']
Num active:  2


Before using the filter it is recommended to understand what it does. If not this step can be skipped. The filtering can scale badly with number of calculations run. 

In [2]:
workflow.filter_active_MD(tol=0.015)

Progress: 90%

Filter reduced configuration space from 10 to 9
Active configs updated.


### Part III

now ready to set up vasp calculations:

In [3]:
workflow.build_QM_calculations('./vasp_base_PuO2',label='VASP')

Number of active configs = number of QM calculation directories = 9
Calculations stored in ./QM_calculations


Now run DFT calculations, since these calculations can often take a large amount of disk space, mlipts has the option to save relevant and delete heavy files ```save_and_remove=True```.

Be cautious if you plan to run many partitions, for example above 64 archer2 will prevent additional submissions.

In [ ]:

vasp_exec = '/vasp.6.5.0/bin/vasp_ncl' # replace with path to vasp exec.
vasp_cmd_line = f'srun --hint=nomultithread --distribution=block:block {vasp_exec}'

# for save and remove
python_env = '/path/to/python_env' # replace with python enviroment with mlipts installed. 
database_file = 'training_data.xyz'

workflow.write_QM_submission_scripts(
    QM_cmd_line=vasp_cmd_line,
    nodes=1,ranks=128,time='02:30:00',
    npartitions=3,
    hpc='archer2',
    hpc_account='my_account',
    python_env=python_env,
    database_file=database_file,
    submit=True,
)

MD submission script saved to: ./QMscripts/QM_submission_script_#0
Submitted batch job 11868803
MD submission script saved to: ./QMscripts/QM_submission_script_#1
Submitted batch job 11868804
MD submission script saved to: ./QMscripts/QM_submission_script_#2
Submitted batch job 11868805


The data is formatted for MACE and can be manipulated using ASE.